In [1]:
from dpt_run import *

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
config = load_config('config_2.yaml')

# create trajectories dataset
if not os.path.exists(config["trajectories_path"]):
    problem_class = getattr(p, config["problem"])
    problems = [problem_class(**config["problem_params"], seed=i) for i in range(config["n_problems"])]
    problems2trajectories(problems=problems, save_dir=config["trajectories_path"])

dataset = MarkovianOfflineDataset(config["trajectories_path"], seq_len=config["model_params"]["seq_len"])
dataloader = DataLoader(dataset=dataset, batch_size=config["batch_size"], num_workers=config["num_workers"])

In [27]:
config = load_config('config_2.yaml')
config = load_config('config.yaml')

seeds = range(10)

trajectories_path = os.path.join(config["trajectories_path"], config["problem"])
results_path = os.path.join(config["results_path"], config["problem"])
if not os.path.exists(trajectories_path):
    if not os.path.exists(results_path):
        problem_class = getattr(p, config["problem"])
        problems = [problem_class(**config["problem_params"], seed=i) for i in seeds]
        problems2trajectories(problems=problems, save_dir=trajectories_path)
    else:
        results2trajectories(read_dir=results_path, save_dir=trajectories_path)

dataset = MarkovianOfflineDataset(trajectories_path, seq_len=config["model_params"]["seq_len"])
dataloader = DataLoader(dataset=dataset, batch_size=config["batch_size"], num_workers=config["num_workers"])

In [3]:
import torch

In [4]:
# !python dpt_run.py

In [5]:
import problems as p
from dpt_run import problems2trajectories

In [6]:
config = load_config('config_2.yaml')

problem_class = getattr(p, config["problem"])
n_t = config["n_train_problems"]
n_v = config["n_val_problems"]
seeds_t = range(0, n_t)
seeds_v = range(n_t, n_t + n_v)

train_offline_dataloader = get_offline_dataloader(config["trajectories_path"], config)
print('train_offline_dataloader:', len(train_offline_dataloader.dataset))

train_offline_dataloader: 10


In [7]:
batch = next(iter(train_offline_dataloader))
for key, val in batch.items():
    print(f'{key:>13} {list(val.shape)}')

  query_state [8, 1]
       states [8, 50, 1]
      actions [8, 50]
  next_states [8, 50, 1]
      rewards [8, 50]
target_action [8]


In [8]:
model = DPTSolver(config)
model.eval()
model._offline_step(batch, 0)

offline target_action torch.Size([8])
torch.Size([8, 1024]) torch.Size([8])
torch.Size([8, 1024]) torch.Size([8])


{'loss': tensor(6.9278, device='cuda:0', grad_fn=<NllLossBackward0>),
 'accuracy': tensor(0., device='cuda:0')}

In [9]:
# validation online dataset
problem_class = getattr(problems, config["problem"])
val_problems = [problem_class(**config["problem_params"], seed=i) for i in config["val_seeds"]]
val_online_dataloader = get_online_dataloader(problems=val_problems, config=config)
print('val_online_dataloader:', len(val_online_dataloader.dataset))

TypeError: get_online_dataloader() got an unexpected keyword argument 'problems'

In [8]:
batch = next(iter(val_online_dataloader))
batch

[{'query_state': tensor([1.2332]),
  'transition_function': functools.partial(<function transition_function at 0x7375bae51040>, problem=<problems.net.Net object at 0x7375ec762d60>),
  'reward_function': functools.partial(<function reward_function at 0x7375bae510d0>, problem=<problems.net.Net object at 0x7375ec762d60>),
  'target_action': tensor(6)}]

In [9]:
model._online_step(batch, 0)

{'loss': tensor(2.7789), 'accuracy': tensor(0.1000)}

In [ ]:



# # logger = WandbLogger(**config["wandb_params"])
# model = DPTSolver(config)

# trainer = L.Trainer(
#     # logger=logger,
#     precision=config["precision"],
#     max_epochs=config["max_epochs"],
#     log_every_n_steps=config["log_every_n_steps"],
#     default_root_dir=config["wandb_params"]["save_dir"],
#     enable_model_summary=False
#     # deterministic=True
# )
# trainer.fit(
#     model=model, 
#     train_dataloaders=train_offline_dataloader, 
#     val_dataloaders=[val_offline_dataloader, val_online_dataloader]
# )

train_offline_dataloader: 16200


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/j/miniconda3/envs/DPT/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


val_offline_dataloader: 600
val_online_dataloader: 10


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

### DPT Net

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import lightning as L

from solvers.dpt.train import DPTSolver
from problems import Net
from dpt_run import get_online_dataloader, load_config
from utils import print_trajectory

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from problems import Net
# from utils import *

# problem = Net(d=4, n=2, seed=0)
# all_actions = get_xaxis(d=problem.d, n=problem.n)
# all_states = problem.target(all_actions)
# print(np.array([all_states.max()]).shape)
# print(np.empty(shape=(1)).shape)

In [ ]:
# logits = torch.randn(size=(1, 10))
# # plt.plot(logits[0])

# for t in (0.01, 0.5, 1.0, 1.5, 2.0, 5.0, 100.0):
#     probs_t = torch.nn.functional.softmax(logits / t, dim=-1)
#     plt.plot(probs_t[0], label=t)

# plt.legend()
# plt.show()

In [3]:
config = load_config('config.yaml')

d = int(np.log2(config["model_params"]["num_actions"]))
problems = [Net(d=d, n=2, seed=i) for i in range(271, 281)]
val_online_dataloader = get_online_dataloader(problems, config)
print('val_online_dataloader:', len(val_online_dataloader.dataset))

val_online_dataloader: 10


In [4]:
checkpoint_file = '../GreyBoxDPTOptimizerData/GreyBoxDPTOptimizer/0cxrd4bf/checkpoints/epoch=299-step=9600.ckpt'
model = DPTSolver.load_from_checkpoint(checkpoint_file).to(device)
model.eval()
print()

In [5]:
def check_temperature(temperature_function):
    visited_points = []
    convergency_step = []

    for _ in range(10):
        for batch in val_online_dataloader:
            for sample in batch:
                result = model.run(
                    **sample,
                    n_steps=config["model_params"]["seq_len"],
                    return_trajectory=True,
                    temperature_function=temperature_function
                )
                right_action_indexes = torch.where(result["actions"] == sample["target_action"])[0]
                if len(right_action_indexes):
                    right_action_step = right_action_indexes[0].item() + 1
                    visited_points_number = len(torch.unique(result["actions"][:right_action_step]))
                else:
                    right_action_step = np.nan
                    visited_points_number = np.nan
                convergency_step.append(right_action_step)
                visited_points.append(visited_points_number)
            # print(model._online_step(batch, 0))
            # break
    mask = np.isnan(convergency_step)
    if mask.all():
        convergency_step = np.nan
        visited_points = np.nan
    else:
        convergency_step = np.nanmean(convergency_step)
        visited_points = np.nanmean(visited_points)
    return convergency_step, visited_points, mask

In [ ]:
temperature_results = {}
temperature_functions = {
   'argmax': lambda x: 0,
    't = 0.00001': lambda x: 0.00001,
    't = 0.5': lambda x: 0.5,
    't = 1.0': lambda x: 1.0, 
    't = 10':  lambda x: 10,
    't = 100': lambda x: 100,
    't ~ 1 - n_step / n_steps': lambda x: 1 - x / config["model_params"]["seq_len"],
    't ~ 10 * (1 - n_step / n_steps)': lambda x: 10 * (1 - x / config["model_params"]["seq_len"]),
    't ~ 100 * (1 - n_step / n_steps)': lambda x: 100 * (1 - x / config["model_params"]["seq_len"]) 
}
for name, temperature_function in temperature_functions.items():
   convergency_step, visited_points, mask = check_temperature(temperature_function)
   temperature_results[name] = dict(
      convergency_step=convergency_step, 
      visited_points=visited_points, 
      mask=mask,
      temperature_function=temperature_function
    ) 

In [ ]:
seq_len = config["model_params"]["seq_len"]
n_step_range = np.arange(0, seq_len)
num_actions = config["model_params"]["num_actions"]

plt.figure(figsize=(10, 4))
for name, result in temperature_results.items():
    print(f'Temperature function: {name}')
    if result['mask'].all():
        print('Has not converged')
    else:
        s_1 = f'Converged at {result["convergency_step"]:.1f}/{seq_len} step'
        s_2 = f' in {np.sum(~result["mask"])}/{len(result["mask"])} cases'
        s_3 = f' and visited {result["visited_points"]:.1f}/{num_actions} points'
        print(s_1 + s_2 + s_3)
    print()
    plt.plot(n_step_range, [result["temperature_function"](n_step) for n_step in n_step_range], label=name)

plt.hlines(0, n_step_range[0], n_step_range[-1])
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
# print_trajectory(result, problems[0])

In [ ]:
# trainer = L.Trainer(
#     precision=config["precision"],
#     enable_model_summary=False
# )
# trainer.test(model, dataloaders=val_online_dataloader)

In [ ]:
# checkpoint = torch.load(checkpoint_file)
# print(checkpoint['hyper_parameters'])

###  DPT Quadratic

In [10]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import lightning as L

from solvers.dpt.train import DPTSolver
from dpt_run_2 import *
# from dpt_run import *

from problems import Quadratic, Net
from utils import print_trajectory

device = "cuda" if torch.cuda.is_available() else "cpu"

In [35]:
!python dpt_run_2.py

Seed set to 42
[[0]
 [1]
 [2]
 [3]
 [4]] [[453]
 [453]
 [453]
 [453]
 [453]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[608]
 [608]
 [608]
 [608]
 [608]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[1023]
 [1023]
 [1023]
 [1023]
 [1023]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[449]
 [449]
 [449]
 [449]
 [449]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[0]
 [0]
 [0]
 [0]
 [0]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[703]
 [703]
 [703]
 [703]
 [703]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[0]
 [0]
 [0]
 [0]
 [0]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[582]
 [582]
 [582]
 [582]
 [582]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[0]
 [0]
 [0]
 [0]
 [0]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[1023]
 [1023]
 [1023]
 [1023]
 [1023]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[374]
 [374]
 [374]
 [374]
 [374]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[553]
 [553]
 [553]
 [553]
 [553]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[880]
 [880]
 [880]
 [880]
 [880]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[0]
 [0]
 [0]
 [0]
 [0]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[498]
 [498]
 [498]
 [498]
 [498]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[0]
 [0]
 [0]
 [0]
 [0]]
[[0]
 [1]
 [2]
 [3]
 [4]] [[1023]

In [2]:
%load_ext autoreload
%autoreload 2

In [18]:
from utils import *
from dpt_run_2 import *

config = load_config('config.yaml')

train_offline_dataloader = get_offline_dataloader(config["train_histories_path"], config)
val_offline_dataloader = get_offline_dataloader(config["val_histories_path"], config)
print('train_offline_dataloader:', len(train_offline_dataloader.dataset))
print('val_offline_dataloader:', len(val_offline_dataloader.dataset))

# d = int(np.log2(config["model_params"]["num_actions"]))
# val_online_dataloader_1 = get_online_dataloader([Net(d=d, n=2, seed=i) for i in range(271, 281)], config)
# val_online_dataloader_2 = get_online_dataloader([Net(d=d, n=2, seed=i) for i in range(1, 11)], config)
# print('val_online_dataloader_1:', len(val_online_dataloader_1.dataset))
# print('val_online_dataloader_2:', len(val_online_dataloader_2.dataset))

# logger = WandbLogger(**config["wandb_params"])
# model = DPTSolver(config)

# trainer = L.Trainer(
#     logger=logger,
#     precision=config["precision"],
#     max_epochs=config["max_epochs"],
#     log_every_n_steps=config["log_every_n_steps"],
#     default_root_dir=config["wandb_params"]["save_dir"],
#     enable_model_summary=False
#     # deterministic=True
# )
# trainer.fit(
#     model=model, 
#     train_dataloaders=train_offline_dataloader, 
#     val_dataloaders=[val_offline_dataloader,]
# )

train_offline_dataloader: 1000
val_offline_dataloader: 100


In [19]:
batch = next(iter(train_offline_dataloader))

In [28]:
batch['target_action'].shape

torch.Size([16])